In [ ]:
!pip install opencv-python matplotlib numpy

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# função para exibir imagens
def show_img(img, title="Imagem"):
    plt.figure(figsize=(6,6))
    if len(img.shape) == 2:  # grayscale
        plt.imshow(img, cmap="gray")
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()

In [ ]:
# função para exibir histograma
def hist_img(img, title="Imagem"):
    plt.subplot(1, 2, 2)
    
    # Se for escala de cinza
    if len(img.shape) == 2:
        # .ravel() "achata" a matriz 2D para um vetor 1D
        plt.hist(img.ravel(), bins=256, range=[0, 256], color='gray')
        plt.title("Histograma (Escala de Cinza)")
    
    # Se for colorida
    elif len(img.shape) == 3:
        cores = ('b', 'g', 'r')
        legendas = ('Canal Azul', 'Canal Verde', 'Canal Vermelho')
        
        for i, cor in enumerate(cores):
            # Calcula o histograma para o canal 'i'
            hist = cv2.calcHist([img], [i], None, [256], [0, 256])
            plt.plot(hist, color=cor, label=legendas[i])
        
        plt.title("Histograma Colorido (B, G, R)")
        plt.legend()
    plt.xlabel("Intensidade do Pixel")
    plt.ylabel("Número de Pixels")
    plt.xlim([-10, 265])
    plt.tight_layout()
    plt.show()

In [ ]:
# carregar a imagem
img = cv2.imread('sininho.jpeg')
show_img(img, "Imagem Original")

In [ ]:
# converta para escala de cinza
img_cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
show_img(img_cinza, "Imagem Original - Escala de Cinza")
hist_img(img_cinza, "Histogram Imagem")
hist_img(img, "Histogram Imagem RGB")

In [ ]:
# A imagem original tem shape (altura, largura, 3).
# Precisamos "achatar" (reshape) para (altura * largura, 3).
img_data = img.reshape((-1, 3))
img_data_cinza = img_cinza.reshape((-1, 3))
# Converter para np.float32, pois o kmeans exige isso
img_data = np.float32(img_data)
img_data_cinza = np.float32(img_data_cinza)

In [ ]:
# numero de clusters (cores)
K = 7

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1.0)
attempts = 10

# cv2.kmeans(data, K, bestLabels, criteria, attempts, flags)
compactness, labels, centers = cv2.kmeans(img_data, K, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)

# Reconstruir a imagem quantizada colorida
centers = np.uint8(centers)
img_quantizada_flat = centers[labels.flatten()]
img_quantizada = img_quantizada_flat.reshape(img.shape)

# cv2.kmeans(data, K, bestLabels, criteria, attempts, flags)
compactness, labels, centers = cv2.kmeans(img_data_cinza, K, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)

# Reconstruir a imagem quantizada em escala de cinza
centers = np.uint8(centers)
img_quantizada_flat = centers[labels.flatten()]
img_quantizada_cinza = img_quantizada_flat.reshape(img_cinza.shape)

# Exibir resultados
show_img(img_quantizada_cinza, title=f"K-Means com K={K} cores")
show_img(img_quantizada, title=f"K-Means com K={K} cores")